In [1]:
import numpy as np
import pandas as pd
import os
import csv
import re
from dotenv import load_dotenv
import json
from openai import OpenAI
from urllib.parse import urlparse
import hashlib
import pdfkit
import requests
from playwright.sync_api import sync_playwright
from openai import OpenAI
import json
from bs4 import BeautifulSoup
from urllib.parse import quote_plus
from itertools import product
import traceback

In [2]:
path = '/home/cptaswadu/RESCUE-n8n/insurance'
load_dotenv(dotenv_path=os.path.join(path, ".env"))
openai_api_key = os.getenv("OPEN_AI_API_KEY")
perplexity_api_key = os.getenv("PERPLEXITY_API_KEY")
chatgpt_client = OpenAI(api_key=openai_api_key)

In [3]:
def find_policy_url_with_web_search(patient_info_text, model="chatgpt", chatgpt_client=None, perplexity_api_key=None, max_retries=3):
    prompt = f"""
You are a clinical insurance assistant.
Your task is to search for the most relevant and up-to-date official insurance policy document URL that addresses **genetic testing coverage** for the following patient. 
Take into account both the patient’s clinical condition and insurance plan.
Please focus on the insurance provider and the plan, genetic test, and the state where the patient resides based on the patient information.

PATIENT INFORMATION:
{patient_info_text}

Instructions:
- Retrieve a **direct URL** to the official insurance policy that best fits this specific clinical scenario and insurance plan.
- Emphasize documents that discuss **medical necessity**, **clinical guidelines**, **prior authorization requirements**, and **coverage criteria**.
- If no relevant policy is found, respond with: **"No policy found."**
- Output **only the URL**. Do not include explanations, summaries, or any additional content.
"""

    messages = [
        {"role": "system", "content": "You are a clinical insurance assistant."},
        {"role": "user", "content": prompt}
    ]

    def call_chatgpt():
        response = chatgpt_client.responses.create(
            model="gpt-4o",
            input=messages,
            tools=[{"type": "web_search_preview"}]
        )
        return response.output_text.strip()

    def call_perplexity():
        headers = {
            "Authorization": f"Bearer {perplexity_api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": "sonar-pro",
            "messages": messages,
            "temperature": 0
        }
        url = "https://api.perplexity.ai/chat/completions"
        res = requests.post(url, headers=headers, json=data)
        if res.status_code == 200:
            return res.json()["choices"][0]["message"]["content"].strip()
        else:
            raise Exception(f"Perplexity error: {res.status_code} - {res.text}")

    for attempt in range(1, max_retries + 1):
        try:
            print(f"🔁 Attempt {attempt} ({model})...")
            result_text = call_perplexity() if model == "perplexity" else call_chatgpt()
            print("🔎 Raw result from LLM:\n", result_text)
            if not result_text.startswith("http"):
                return "No policy found."
            return result_text
        except Exception as e:
            print(f"❌ Attempt {attempt} failed: {e}")

    return "No policy found."


In [4]:
def clean_json_response(response_text):
    original = response_text.strip()

    # Step 0: Check for hallucinated greeting (Perplexity fallback)
    if "how can I assist you" in original.lower() or "insurance-related questions" in original.lower():
        raise ValueError("Perplexity returned generic assistant response instead of JSON.")

    # Step 1: Try direct parsing
    try:
        return json.loads(original)
    except json.JSONDecodeError:
        pass

    # Step 2: Remove code block wrappers
    cleaned = re.sub(r"^```(?:json)?\s*|\s*```$", "", original, flags=re.IGNORECASE).strip()
    try:
        return json.loads(cleaned)
    except json.JSONDecodeError:
        pass

    # Step 3: Try to extract the first {...} JSON-like block
    match = re.search(r"(\{[\s\S]*?\})", original)
    if match:
        try:
            return json.loads(match.group(1))
        except json.JSONDecodeError:
            pass

    raise ValueError("No valid JSON found in the response.")

In [5]:
def format_question_block(q, indent=2):
    indent_str = " " * indent
    question_line = f"{q['question']}"

    if q.get("options") == ["Free text answer"]:
        question_line += f"\n{indent_str}(Free text answer allowed.)"
    else:
        question_line += f"\n{indent_str}Options: {', '.join(q['options'])}"

        if "additional_if_yes" in q:
            question_line += f"\n{indent_str}  If 'Yes':"
            for item in q["additional_if_yes"]:
                if isinstance(item, str):
                    question_line += f"\n{indent_str}    - {item}"
                elif isinstance(item, dict):
                    question_line += f"\n{indent_str}    - {format_question_block(item, indent + 6)}"

        if "additional_if_no" in q:
            question_line += f"\n{indent_str}  If 'No':"
            for item in q["additional_if_no"]:
                if isinstance(item, str):
                    question_line += f"\n{indent_str}    - {item}"
                elif isinstance(item, dict):
                    question_line += f"\n{indent_str}    - {format_question_block(item, indent + 6)}"

    return question_line


def format_questions(questions_list):
    return "\n\n".join([
        f"{q['id']}. {format_question_block(q)}"
        for q in questions_list
    ])


In [6]:
def run_qna(
    case_id,
    patient_info_text,
    policy_url,
    questions_list,
    qna_model="chatgpt", 
    chatgpt_client=None,
    perplexity_api_key=None,
    search_model="chatgpt"
):
    questions_formatted = format_questions(questions_list)

    prompt = f"""
You are a clinical insurance assistant specializing in genetic testing coverage policies.

You will be given:
1. Patient clinical information 
2. The official policy document URL

Your task is to read and interpret the policy content at the provided URL,
and answer the following clinical coverage questions for the patient based strictly on that document and the patient's information.

Focus especially on factors such as:
- Age requirements
- Clinical guidelines
- Medical necessity
- Prior authorization criteria 
- Any other relevant rules related to genetic testing coverage for this patient.

Instructions:
- If the policy document clearly answers the question, follow its criteria.
- If the policy is unclear, cautiously use the patient’s clinical information to infer a reasonable answer.
- DO NOT rely on general knowledge or assumptions beyond the given document.
- Strictly follow question constraints:
    - Choose only from the provided options.
    - For free text questions, answer concisely and clearly.
    - If a question says "If Yes, ALSO select from ...", and you answer "Yes", you MUST also select follow-up items.
    - Same for "If No, ALSO select from ...".

Return your response in JSON format ONLY.

==== PATIENT INFORMATION ====
{patient_info_text}

==== POLICY DOCUMENT URL ====
{policy_url}

==== QUESTIONS ====
{questions_formatted}

Output your answers in JSON format only.
"""

    messages = [
        {"role": "system", "content": "You are a clinical insurance assistant."},
        {"role": "user", "content": prompt}
    ]

    print("📨 Prompt sent to model:\n", prompt)

    def call_chatgpt():
        response = chatgpt_client.responses.create(
            model="gpt-4o",
            input=messages,
            tools=[{"type": "web_search_preview"}]
        )
        return response.output_text.strip()

    def call_perplexity():
        headers = {
            "Authorization": f"Bearer {perplexity_api_key}",
            "Content-Type": "application/json"
        }
        data = {
            "model": "sonar-pro",
            "messages": messages,
            "temperature": 0
        }
        url = "https://api.perplexity.ai/chat/completions"
        res = requests.post(url, headers=headers, json=data)
        if res.status_code == 200:
            return res.json()["choices"][0]["message"]["content"].strip()
        else:
            raise Exception(f"Perplexity error: {res.status_code} - {res.text}")

    try:
        print(f"🧠 Running QnA ({qna_model})...")
        result_content = call_perplexity() if qna_model == "perplexity" else call_chatgpt()
        print("📝 Raw LLM output:\n", result_content)
        result_json = clean_json_response(result_content)

        final_result = {}
        for k, v in result_json.items():
            if k == "policy_url":
                continue
            if "_selection" in k or "_details" in k:
                base_key = k.replace("_selection", "").replace("_details", "")
                final_result[f"{base_key}_followup"] = [v] if isinstance(v, str) else v
            else:
                final_result[k] = v

        result_dir = f"/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/{search_model}_{qna_model}"
        os.makedirs(result_dir, exist_ok=True)
        filename = os.path.join(result_dir, f"{case_id}_qna_result.json")

        with open(filename, "w") as f:
            json.dump(final_result, f, indent=2)

        print(f"✅ QnA result saved to {filename}")

    except Exception as e:
        print("❗ JSON parsing error:", e)
        final_result = {
            "error": "JSON parsing failed",
            "raw_content": result_content
        }

    print("QnA Result JSON:", final_result)
    return final_result


In [7]:
questions_file_path = "/home/cptaswadu/RESCUE-n8n/insurance/dataset/Insurance_Genetic_Testing_QA.json"

with open(questions_file_path, "r") as f:
    questions_data = json.load(f)

questions_list = questions_data["questions"]

In [8]:
case_ex = [
    {
        "id": "Case1",
        "patient_info": "An 8-year-old boy with neurodevelopmental delay and seizures. A prior chromosomal microarray test was negative. Whole exome sequencing (WES) has been requested by the genetic counselor to investigate potential underlying genetic causes that may guide diagnosis and future treatment decisions. There is also a family history of neurodevelopmental disorders, as his older brother was diagnosed with autism spectrum disorder. The patient is covered by United Healthcare Choice Plus through a family plan and resides in New Jersey."
    }
]


In [9]:
def run_all_model_combinations(case_ex, questions_list, chatgpt_client, perplexity_api_key):
    model_options = ["chatgpt", "perplexity"]
    combinations = list(product(model_options, repeat=2))

    for search_model, qna_model in combinations:
        print(f"\n🚀 Starting experiments for: {search_model}_{qna_model}\n")

        for case in case_ex:
            case_id = case["id"]
            patient_info = case["patient_info"]

            print(f"\n=== Running for {case_id} ===")

            try:
                policy_url = find_policy_url_with_web_search(
                    patient_info_text=patient_info,
                    model=search_model,
                    chatgpt_client=chatgpt_client,
                    perplexity_api_key=perplexity_api_key
                )

                run_qna(
                    case_id=case_id,
                    patient_info_text=patient_info,
                    policy_url=policy_url,
                    questions_list=questions_list,
                    qna_model=qna_model,
                    chatgpt_client=chatgpt_client,  
                    perplexity_api_key=perplexity_api_key,
                    search_model=search_model
                )

            except Exception as e:
                print(f"❌ Failed for {case_id} in {search_model}_{qna_model}")
                traceback.print_exc()

run_all_model_combinations(
    case_ex=case_ex,
    questions_list=questions_list,
    chatgpt_client=chatgpt_client,  
    perplexity_api_key=perplexity_api_key
)



🚀 Starting experiments for: chatgpt_chatgpt


=== Running for Case1 ===
🔁 Attempt 1 (chatgpt)...
🔎 Raw result from LLM:
 https://www.uhcprovider.com/en/prior-auth-advance-notification/genetic-molecular-lab.html
📨 Prompt sent to model:
 
You are a clinical insurance assistant specializing in genetic testing coverage policies.

You will be given:
1. Patient clinical information 
2. The official policy document URL

Your task is to read and interpret the policy content at the provided URL,
and answer the following clinical coverage questions for the patient based strictly on that document and the patient's information.

Focus especially on factors such as:
- Age requirements
- Clinical guidelines
- Medical necessity
- Prior authorization criteria 
- Any other relevant rules related to genetic testing coverage for this patient.

Instructions:
- If the policy document clearly answers the question, follow its criteria.
- If the policy is unclear, cautiously use the patient’s clinical inform

In [10]:
def merge_qna_jsons_to_csv(folder_path, output_csv_path):
    all_data = []

    for file in os.listdir(folder_path):
        if file.endswith("_qna_result.json"):
            case_id = file.replace("_qna_result.json", "")
            json_path = os.path.join(folder_path, file)

            with open(json_path, "r") as f:
                try:
                    result = json.load(f)
                    flat_result = {"case_id": case_id}

                    for k, v in result.items():
                        if isinstance(v, list):
                            flat_result[k] = "; ".join(map(str, v))
                        else:
                            flat_result[k] = v

                    all_data.append(flat_result)
                except Exception as e:
                    print(f"❗ Failed to parse {file}: {e}")

    if all_data:
        df = pd.DataFrame(all_data)
        df.to_csv(output_csv_path, index=False)
        print(f"✅ Merged CSV saved to: {output_csv_path}")
    else:
        print(f"⚠️ No valid QnA result files found in: {folder_path}")

def merge_all_combinations_to_csv(base_dir):
    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            output_csv = os.path.join(base_dir, f"{folder_name}.csv")
            merge_qna_jsons_to_csv(folder_path, output_csv)


merge_all_combinations_to_csv(
    base_dir="/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End"
)

✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/chatgpt_perplexity.csv
✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/chatgpt_chatgpt.csv
✅ Merged CSV saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/perplexity_perplexity.csv


In [11]:
def load_converted_results_from_folder(folder_path):
    converted_results = {}
    for file in os.listdir(folder_path):
        if file.endswith("_qna_result.json"):
            case_id = file.replace("_qna_result.json", "")
            file_path = os.path.join(folder_path, file)
            with open(file_path, "r") as f:
                try:
                    data = json.load(f)
                except json.JSONDecodeError as e:
                    print(f"❗ JSON decode error in {file}: {e}")
                    continue

                clean = {}
                for k, v in data.items():
                    if k.endswith("_followup"):
                        clean[k] = v if isinstance(v, list) else [v]
                    else:
                        clean[k] = v

                converted_results[case_id] = clean
    return converted_results

In [12]:
ground_truth = {
    "Case1": {
        "Q0": "Whole Exome Sequencing (WES)",
        "Q1": "Yes",
        "Q2": "Yes",
        "Q3": "Yes",
        "Q4": "Yes",
        "Q4_followup": [
            {
                "answer": "Yes",
                "followup": ["ACMG"]
            }
        ],
        "Q5": "Yes",
        "Q6": "No",
        "Q7": "Yes",
        "Q8": "No",
        "Q9": "Yes",
        "Q9_followup": ["Yes"],
        "Q10": "No",
        "Q10_followup": ["Diagnostic"],
        "Q11": "Yes",
        "Q12": "No",
        "Q13": "Yes",
        "Q14": "Yes",
        "Q14_followup": ["81415", "81416"],
        "Q15": "No",
        "Q16": "Yes"
    }
}

In [13]:
def evaluate_qna_result(case_id, predicted_result, gold_result, folder_path=None):
    records = []
    correct_count = 0
    total_count = 0

    for qid in gold_result:
        if not qid.startswith("Q") or qid == "policy_url" or qid == "Q17" or "_followup" in qid:
            continue

        pred_answer = predicted_result.get(qid, "")
        gold_answer = gold_result.get(qid, "")

        if isinstance(pred_answer, list):
            pred_answer = ", ".join(pred_answer)
        if isinstance(gold_answer, list):
            gold_answer = ", ".join(gold_answer)

        pred_answer = pred_answer.strip()
        gold_answer = gold_answer.strip()

        is_correct = pred_answer == gold_answer
        score = 1 if is_correct else 0

        records.append({
            "Case": case_id,
            "Question": qid,
            "Predicted": pred_answer,
            "Gold": gold_answer,
            "Score": score
        })

        total_count += 1
        correct_count += score

        followup_key = qid + "_followup"
        pred_followup = predicted_result.get(followup_key, None)
        gold_followup = gold_result.get(followup_key, None)

        if is_correct and gold_followup is not None:
            def normalize(ans):
                if ans is None:
                    return "None"
                if isinstance(ans, list):
                    return ", ".join([a if isinstance(a, str) else a.get("answer", str(a)) for a in ans])
                return ans if isinstance(ans, str) else str(ans)

            pred_followup_norm = normalize(pred_followup)
            gold_followup_norm = normalize(gold_followup)

            followup_score = 1 if pred_followup_norm == gold_followup_norm else 0

            records.append({
                "Case": case_id,
                "Question": followup_key,
                "Predicted": pred_followup_norm,
                "Gold": gold_followup_norm,
                "Score": followup_score
            })

            total_count += 1
            correct_count += followup_score

    accuracy = correct_count / total_count * 100 if total_count > 0 else 0

    records.append({
        "Case": case_id,
        "Question": "TOTAL",
        "Predicted": f"Correct: {correct_count}",
        "Gold": f"Incorrect: {total_count - correct_count}",
        "Score": f"Accuracy: {accuracy:.2f}%"
    })

    df = pd.DataFrame(records)

    # ✅ Save CSV if folder_path is given
    if folder_path:
        os.makedirs(folder_path, exist_ok=True)
        csv_path = os.path.join(folder_path, f"evaluation_{case_id}.csv")
        df.to_csv(csv_path, index=False)
        print(f"📄 Saved evaluation to {csv_path}")

    return df, accuracy

In [14]:
def evaluate_all_folders_with_summary(base_dir, gold_answers, summary_output_csv):
    eval_output_dir = os.path.join(base_dir, "Evaluation")
    os.makedirs(eval_output_dir, exist_ok=True)

    summary_records = []

    for folder_name in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        print(f"\n📊 Evaluating folder: {folder_name}")

        converted_results = load_converted_results_from_folder(folder_path)
        all_dfs = []
        accuracies = []

        for case_id, pred_result in converted_results.items():
            gold_result = gold_answers.get(case_id)
            if gold_result is None:
                continue

            df_case, acc = evaluate_qna_result(case_id, pred_result, gold_result)
            all_dfs.append(df_case)
            accuracies.append(acc)

        if all_dfs:
            merged_df = pd.concat(all_dfs, ignore_index=True)
            eval_csv_path = os.path.join(eval_output_dir, f"{folder_name}.csv")
            merged_df.to_csv(eval_csv_path, index=False)
            print(f"✅ Saved: {eval_csv_path}")

            
            summary_records.append({
                "Model_Combination": folder_name,
                "Mean_Accuracy": f"{sum(accuracies)/len(accuracies):.2f}%" if accuracies else "N/A"
            })

    
    if summary_records:
        summary_df = pd.DataFrame(summary_records)
        os.makedirs(os.path.dirname(summary_output_csv), exist_ok=True)
        summary_df.to_csv(summary_output_csv, index=False)
        print(f"\n✅ Summary saved to: {summary_output_csv}")
        print(summary_df)

In [15]:
evaluate_all_folders_with_summary(
    base_dir="/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End",
    gold_answers=ground_truth,
    summary_output_csv="/home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/Evaluation/summary_accuracy.csv"
)



📊 Evaluating folder: chatgpt_perplexity
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/Evaluation/chatgpt_perplexity.csv

📊 Evaluating folder: Evaluation

📊 Evaluating folder: chatgpt_chatgpt
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/Evaluation/chatgpt_chatgpt.csv

📊 Evaluating folder: perplexity_perplexity
✅ Saved: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/Evaluation/perplexity_perplexity.csv

✅ Summary saved to: /home/cptaswadu/RESCUE-n8n/insurance/results/LLM_QnA/End-To-End/Evaluation/summary_accuracy.csv
       Model_Combination Mean_Accuracy
0     chatgpt_perplexity        72.22%
1        chatgpt_chatgpt        71.43%
2  perplexity_perplexity        63.16%
